In [1]:
import random
import json
import pandas as pd

#### Loading Snaposhot Serengeti data

In [2]:
with open('../data/SnapshotSerengetiSplits_v0.json') as json_file:
    recommend_train_val_splits = json.load(json_file)

In [3]:
serengeti_annotations = pd.read_csv('../data/SnapshotSerengeti_v2_1_annotations.csv')
serengeti_annotations = serengeti_annotations[['capture_id', 'season', 'site', 'question__species']].copy()

/home/fagner/anaconda3/envs/cameratraps-detector/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
serengeti_images = pd.read_csv('../data/SnapshotSerengeti_v2_1_images.csv')
serengeti_images = serengeti_images.drop('Unnamed: 0', axis=1)

In [5]:
serengeti_images_labeled = pd.merge(serengeti_images, serengeti_annotations, on='capture_id', how='outer')

We will only use seasons 1-6:

In [6]:
serengeti_images_labeled = serengeti_images_labeled[
    serengeti_images_labeled.season.isin(['S1', 'S2', 'S3', 'S4', 'S5', 'S6'])].copy()

#### Remove images with more than one species identified

In [7]:
non_single_spc_instances = serengeti_images_labeled[
    serengeti_images_labeled[['image_path_rel']].duplicated(keep=False)]
non_single_spc_instances = non_single_spc_instances.image_path_rel.unique()

In [8]:
serengeti_images_labeled = serengeti_images_labeled[
    ~serengeti_images_labeled.image_path_rel.isin(non_single_spc_instances)].copy()

#### Mark train/val images:

In [9]:
val_dev = ['D09',
 'J06',
 'C12',
 'B12',
 'F11',
 'C08',
 'E07',
 'O09',
 'Q07',
 'C13',
 'E04',
 'I06',
 'D10',
 'I08',
 'M11',
 'F02',
 'D06',
 'G09',
 'N03',
 'E10',
 'J09',
 'H13',
 'T13']
#val_dev = random.sample(recommend_train_val_splits['splits']['train'], 23)

In [10]:
serengeti_images_labeled_split = serengeti_images_labeled.copy()

In [11]:
def mark_split(row):
    if row['site'] in val_dev:
        return 'val_dev'
    elif row['site'] in recommend_train_val_splits['splits']['train']:
        return 'train'
    else:
        return 'val'

In [12]:
serengeti_images_labeled_split['split'] = serengeti_images_labeled_split.apply(mark_split, axis=1)

In [13]:
pd.crosstab(serengeti_images_labeled_split.question__species, serengeti_images_labeled_split.split)

split,train,val,val_dev
question__species,,,
aardvark,352,162,54
aardwolf,222,75,15
baboon,3734,500,265
batEaredFox,543,168,42
blank,1522420,535832,278594
buffalo,24531,6707,2417
bushbuck,302,37,14
caracal,130,52,1
cheetah,2284,815,303
